In [7]:
# Supondo que os módulos agent.py e groupchat.py estejam disponíveis
from groupchat import GroupChat
from agent import Agent
from groupchatadmin import GroupChatAdmin
from pipeline import PipelineComponent
from pipeline import Pipeline

class HumanInputMessage(PipelineComponent):
    def process(self, data):
        print(data.tail())
        resposta = input("Digite a resposta: ")
        return resposta
    
class HumanInputNextAgent(PipelineComponent):
    def process(self, **kwargs):
        chatgroup = kwargs.get('chatgroup')
        if not chatgroup:
            raise ValueError("Chatgroup is required for HumanInputPipeline")
        
        agents = chatgroup.agentList
        for i, agent in enumerate(agents):
            print(f"{i + 1}: {agent.name}")

        escolha = None
        while escolha is None:
            try:
                escolha_input = input("Digite o número correspondente ao agente: ")
                escolha = int(escolha_input)
                if escolha < 1 or escolha > len(agents):
                    print("Escolha inválida. Tente novamente.")
                    escolha = None
            except ValueError:
                print("Entrada inválida. Por favor, digite um número.")
        
        next_agent = agents[escolha - 1]
        kwargs['next'] = next_agent
        return kwargs

class AgentReplyComponent(PipelineComponent):
    def process(self, **kwargs):
        # Recupera o agente do kwargs
        agent = kwargs.get('next')
        group_chat = kwargs.get('chatgroup')
        messages = group_chat.get_messages()
        if not agent:
            raise ValueError("Agent is required for AgentReplyComponent")

        reply = agent.generate_reply(data=messages)
        print(reply)
        group_chat.add_message(sender_id=agent.agent_id, message=reply)


pipeline_admin = Pipeline([HumanInputNextAgent(), AgentReplyComponent()])

# Setup do ambiente de teste
group_chat = GroupChat()
agent1 = Agent("agent1", "Agente 1", "role1", Pipeline([HumanInputMessage()]))
agent2 = Agent("agent2", "Agente 2", "role2", Pipeline([HumanInputMessage()]))

group_chat.add_agent(agent1)
group_chat.add_agent(agent2)

chat_admin = GroupChatAdmin("admin", "Admin", "admin_role", pipeline_admin, group_chat, "manage_chat")

# Executando o GroupChatAdmin
chat_admin.run()


2023-12-26 07:04:56,646 - groupchatadmin - INFO - Chat Admin started.


1: Agente 1
2: Agente 2
Empty DataFrame
Columns: [id, sender_id, message, timestamp]
Index: []
OLA
1: Agente 1
2: Agente 2
   id sender_id message                   timestamp
0   1    agent1     OLA  2023-12-26 07:05:02.095533
OLA 2
1: Agente 1
2: Agente 2
   id sender_id message                   timestamp
0   1    agent1     OLA  2023-12-26 07:05:02.095533
1   2    agent2   OLA 2  2023-12-26 07:05:06.919376
AD
1: Agente 1
2: Agente 2
   id sender_id message                   timestamp
0   1    agent1     OLA  2023-12-26 07:05:02.095533
1   2    agent2   OLA 2  2023-12-26 07:05:06.919376
2   3    agent1      AD  2023-12-26 07:05:11.104556
12
1: Agente 1
2: Agente 2
Entrada inválida. Por favor, digite um número.
   id sender_id message                   timestamp
0   1    agent1     OLA  2023-12-26 07:05:02.095533
1   2    agent2   OLA 2  2023-12-26 07:05:06.919376
2   3    agent1      AD  2023-12-26 07:05:11.104556
3   4    agent1      12  2023-12-26 07:05:13.696440


KeyboardInterrupt: Interrupted by user